In [1]:
def getInput():
    return open("../input/day_5.txt").read()

def getInputLines():
    return getInput().split("\n")

# Day 5 Part 1

parsing: /`(^\n?[a-zA-Z\-\:\s]+)((\d+[.\s]?)+)`/gms

In [3]:
import re

text = getInput()

# text = """
# seeds: 79 14 55 13

# seed-to-soil map:
# 50 98 2
# 52 50 48

# soil-to-fertilizer map:
# 0 15 37
# 37 52 2
# 39 0 15

# fertilizer-to-water map:
# 49 53 8
# 0 11 42
# 42 0 7
# 57 7 4

# water-to-light map:
# 88 18 7
# 18 25 70

# light-to-temperature map:
# 45 77 23
# 81 45 19
# 68 64 13

# temperature-to-humidity map:
# 0 69 1
# 1 0 69

# humidity-to-location map:
# 60 56 37
# 56 93 4
# """.strip()

# match all numbers, first match is seed line, subsequent matches are the maps
seeds, *maps = [
    [int(v) for v in re.findall(r"\d+", match.groups()[1], flags=re.MULTILINE)]
    for match in re.finditer(r"(^\n?[a-zA-Z\-\:\s]+)((\d+[.\s]?)+)", text, flags=re.MULTILINE)
    ]
# sort ranges in maps based on their start number
maps = [sorted(r, key=lambda x: x[1]) for r in
    [[m[i:i+3] for i in range(0, len(m), 3)] for m in maps]]

# main
locs = []
for seed in seeds:
    temp = seed
    for map in maps:
        for (dst, src, span) in map:
            if src <= temp < src + span:
                temp = dst + (temp - src)
                break
    locs.append(temp)
min(locs)
# 331445006


331445006

# Day 5 Part 2

- 24092691 too high
- 20522066 too high
- 19624706 too high

In [20]:
import re

text = """
seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
""".strip()

text = getInput()

seeds, *maps = [[int(v) for v in re.findall(r"\d+", match.groups()[1], flags=re.MULTILINE)] for match in re.finditer(r"(^\n?[a-zA-Z\-\:\s]+)((\d+[.\s]?)+)", text, flags=re.MULTILINE)]
seeds = [seeds[i:i+2] for i in range(0, len(seeds), 2)]
maps = [[m[i:i+3] for i in range(0, len(m), 3)] for m in maps]

def split_range(seed_range:tuple[int], rule:tuple[int]):
    pass_through, recycle = [], []
    
    seed_start, seed_length = seed_range
    rule_dst, rule_start, rule_length = rule
    seed_end = seed_start + seed_length - 1
    rule_end = rule_start + rule_length - 1
    rule_shift = rule_dst - rule_start


    if seed_start > rule_end or seed_end < rule_start:
        recycle.append(seed_range)
    elif seed_start >= rule_start and seed_end <= rule_end:
        remapped_start = seed_start + rule_shift
        pass_through.append((remapped_start, seed_length))
    elif seed_start >= rule_start: # seed_end > rule_end
        remapped_start = seed_start + rule_shift
        remapped_length = rule_end - seed_start + 1
        pass_through.append((remapped_start, remapped_length))
        recycled_start = rule_end + 1
        recycled_length = seed_end - recycled_start + 1
        recycle.append((recycled_start, recycled_length))
    elif seed_end <= rule_end: # seed_start < rule_start
        remapped_start = rule_start + rule_shift
        remapped_length = seed_end - rule_start + 1
        pass_through.append((remapped_start, remapped_length))
        recycled_start = seed_start
        recycled_length = (rule_start - 1 ) - seed_start + 1
        recycle.append((recycled_start, recycled_length))
    else: # seed_start < rule_start and seed_end > rule_end
        remapped_start = rule_start
        remapped_length = rule_length
        pass_through.append((remapped_start, remapped_length))
        recycled_start_1 = rule_start
        recycled_length_1 = (rule_start - 1) - seed_start + 1
        recycle.append((recycled_start_1, recycled_length_1))
        recycled_start_2 = rule_end + 1
        recycled_length_2 = seed_end - recycled_start_2 + 1
        recycle.append((recycled_start_2, recycled_length_2))
        
    return pass_through, recycle

for_next_map = [seed for seed in seeds]
for i, rules in enumerate(maps):
    for_next_rule = [seed for seed in for_next_map]
    for_next_map = []
    for rr in rules:
        for_this_rule = [seed for seed in for_next_rule]
        for_next_rule = []
        for sr in for_this_rule:
            pt, rc = split_range(sr, rr)
            for_next_map.extend(pt)
            for_next_rule.extend(rc)
    for_next_map += for_next_rule
locations = filter(lambda x: x[1] > 0, for_next_map)
print("result is:", min(locations, key=lambda x: x[0])[0])


result is: 6472060
